In [19]:
import json
import os

def read_data(datafile=os.path.join(os.getcwd(), '.data', 'master.json')):
    with open(datafile, mode='r') as f:
        lines = [json.loads(l) for l in f.readlines()]
        return lines

data = read_data()
for x in data:
    if x['is_blog'] is True:
        x.update({"is_blog": 1})
    else:
        x.update({"is_blog": 0})
blog_entries = [d for d in data if d['is_blog'] == 1]
news_entries = [d for d in data if d['is_blog'] == 0]
print("Number of Blog Entries: ", len(blog_entries))
print("Number of News Entries: ", len(news_entries))


Number of Blog Entries:  485
Number of News Entries:  1308


In [20]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
from collections import Counter
import numpy as np

init_notebook_mode(connected=True)
kwds = []
[kwds.extend(d['kwds']) for d in data]
kwd_counts = dict(Counter(kwds))
pltdata = [go.Bar(x=[k for k in kwd_counts.keys()], y=[v for v in kwd_counts.values()])]
plot(pltdata)

'file:///home/cmustard/Projects/spyndicated/spyndicated/temp-plot.html'

In [22]:
import numpy as np
from gensim import corpora, models
allids = set(range(len(data)))

training_sample_ids = set(np.random.choice(range(len(data)), 600))
testing_sample_ids = set(np.random.choice([i for i in allids-training_sample_ids], 300))
training_entries = [data[i] for i in training_sample_ids]
testing_entries = [data[j] for j in testing_sample_ids]


In [45]:

testing_kwds = [j['kwds'] for j in testing_entries]
training_kwds = [i['kwds'] for i in training_entries]
# print(testing_kwds[:5])
# print(training_kwds[:5])
d = corpora.Dictionary(training_kwds)
testing_corpus_bows = [d.doc2bow(j) for j in testing_kwds]
training_corpus_bows = [d.doc2bow(i) for i in training_kwds]
lda = models.ldamodel.LdaModel(corpus=training_corpus_bows, num_topics=60)



In [49]:
xt = lda.show_topic(0)
xt[1]


In [50]:
for i in range(10):

    print([(d.id2token[int(x[0])], x[1]) for x in lda.show_topic(i)])
#testing_tops = [lda.get_document_topics(jbow) for jbow in testing_corpus_bows]

#training_tops = [lda.get_document_topics(ibow) for ibow in training_corpus_bows]

# for i in range(len(training_sample_ids)):
#    training_entries[i].update({"topics": training_tops[i]})    
# for j in range(len(testing_sample_ids)):
#    testing_entries[j].update({"topics": testing_tops[j]})
    
        

[('the', 0.046789072), ('to', 0.037078563), ('of', 0.022894038), ('and', 0.019087298), (' ', 0.018943932), ('in', 0.01805378), ('that', 0.012170301), ('for', 0.010497646), ('is', 0.008926169), ('with', 0.008617311)]
[('the', 0.021527715), ('and', 0.010586378), ('to', 0.010497752), ('learning', 0.009277596), ('machine', 0.00927698), ('in', 0.008134774), ('you', 0.0067978585), ('transportation', 0.006549915), ('commerce', 0.0064039775), ('with', 0.0053081596)]
[('the', 0.041114416), ('to', 0.019302355), (' ', 0.015952364), ('and', 0.013408236), ('as', 0.011857941), ('on', 0.011206151), ('of', 0.011138467), ('in', 0.011125654), ('for', 0.009067276), ('pyeongchang', 0.008593416)]
[(' ', 0.056847308), ('the', 0.031264044), ('and', 0.02310892), ('in', 0.018392801), ('for', 0.013221009), ('at', 0.012601764), ('is', 0.009805018), ('of', 0.0072937696), ('to', 0.007167674), ('   ', 0.0066799065)]
[('to', 0.042998634), ('the', 0.040379044), ('of', 0.017852118), (' ', 0.015242027), ('that', 0.0149

In [ ]:
blogkwds = []
blogtags = []
blogtopics = []
newskwds = []
newstags = []
newstopics = []

for j in range(len(testing_entries)):
    ent = testing_entries[j]
    for tag in ent['tags']:
        tt = []
        if type(tag) is list:
            tt.append(tag[0])
        else:
            tt.append(tag)
        if len(tt) > 0:
            ent.update({"tags": tt})
        else:
            pass
        
    if ent['is_blog'] == 1:
        blogkwds.extend([i for i in ent['kwds']])
        blogtags.extend([i.split(",")[0] for i in ent['tags']])
        blogtopics.extend([i[0] for i in ent['topics']])
    else:
        newskwds.extend([i for i in ent['kwds']])
        newstags.extend([i for i in ent['tags']])
        newstopics.extend([i[0] for i in ent['topics']])


In [ ]:
kwd_layout = go.Layout(barmode='overlay', title='Keyword Probabilities by Source Type')
BlogKeywords = go.Histogram(x=blogkwds, name='Blog Keywords', histnorm='probability', opacity=0.75)
NewsKeywords = go.Histogram(x=newskwds, name='News Keywords', histnorm='probability', opacity=0.75)
kwd_traces = [BlogKeywords, NewsKeywords]
kwd_fig = go.Figure(data=kwd_traces, layout=kwd_layout)
iplot(kwd_fig)

tag_layout = go.Layout(barmode='overlay', title='Tag Frequencies by Source Type')
BlogTags = go.Histogram(x=blogtags, name='Blog Tags', opacity=0.75)
NewsTags = go.Histogram(x=newstags, name='News Tags', opacity=0.75)
tag_traces = [BlogTags, NewsTags]
tag_fig = go.Figure(data=tag_traces, layout=tag_layout)
iplot(tag_fig)

topics_layout = go.Layout(barmode='overlay', title='Topic Probabilities by Source Type')
BlogTopics = go.Histogram(x=blogtopics, name='Blog Topics', histnorm='probability', opacity=0.75)
NewsTopics = go.Histogram(x=newstopics, name='News Topics', histnorm='probability', opacity=0.75)
topics_traces = [BlogTopics, NewsTopics]
topics_fig = go.Figure(data=topics_traces, layout=topics_layout)
iplot(topics_fig)

In [ ]:
from sklearn.linear_model import LogisticRegression


